In [1]:
import math
import time

import h5py
import matplotlib.pyplot as plt
import mlflow
import numpy as np
from numpy.random import choice
import torch
import torch.nn as nn
from torch.cuda.amp import autocast
from torch.optim import SGD, Optimizer, Adam, RMSprop
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from tqdm.auto import tqdm as tq
from craterdetection.detection.loss_functions.dice import f_score, BCEDiceLoss, SoftDiceLoss

from craterdetection.detection.deepmoon import DeepMoon
from craterdetection.detection.loss_functions.tversky import TverskyLoss
from craterdetection.detection.unetmini import UNetMini
from craterdetection.detection.training import CraterDataset, RAdam, dice_coefficient

from craterdetection.detection.loss_functions.lovasz import LovaszHingeLoss

from torchsummary import summary

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
mlflow.set_tracking_uri("http://localhost:5000/")
mlflow.set_experiment("crater-detection")

In [3]:
dataset_path = "../data/dataset_nadir.h5"
train_dataset = CraterDataset(dataset_path, "training", device)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)

validation_dataset = CraterDataset(dataset_path, "validation", device)
validation_loader = DataLoader(validation_dataset, batch_size=32, num_workers=0)

test_dataset = CraterDataset(dataset_path, "test", device)
test_loader = DataLoader(test_dataset, batch_size=6, shuffle=True, num_workers=0)

In [4]:
model = DeepMoon(sigmoid=False)
model.load_state_dict(torch.load("../blobs/DeepMoon.pth"))

optimizer = RMSprop(model.parameters(), lr=1e-4, weight_decay=1e-7, momentum=0.4)
# optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=1e-9)
# optimizer = Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)
# optimizer = RAdam(model.parameters(), lr=1e-5)
# loss_function = SoftDiceLoss()
loss_function = nn.BCEWithLogitsLoss()
# loss_function = BCEDiceLoss(lambda_bce=1.0, lambda_dice=1.0)
# loss_function = TverskyLoss(alpha=0.1, beta=0.9)
# loss_function = LovaszHingeLoss()

# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2, verbose=True, cooldown=2)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=1, verbose=True, factor=0.5)
# scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

model.to(device)
loss_function.to(device)

BCEWithLogitsLoss()

In [ ]:
tracked_params = ('lr', 'momentum', 'weight_decay')

num_epochs = 10

name = "Filled" if "filled" in dataset_path else "Edge"

with mlflow.start_run(run_name=name):

    mlflow.log_param('optimizer', type(optimizer).__name__)
    mlflow.log_param('loss_function', type(loss_function).__name__)
    try:
        mlflow.log_param('lambda_dice', loss_function.lambda_dice)
        mlflow.log_param('lambda_bce', loss_function.bce)
    except AttributeError as err:
        pass

    try:
        mlflow.log_param('tversky_alpha', loss_function.alpha)
        mlflow.log_param('tversky_beta', loss_function.beta)
    except AttributeError as err:
        pass

    for tp in tracked_params:
        try:
            mlflow.log_param(tp, optimizer.state_dict()['param_groups'][0][tp])
        except KeyError as err:
            pass

    train_loss_list = []
    valid_loss_list = []
    dice_score_list = []

    for e in range(1, num_epochs+1):
        print(f'\n-----Epoch {e} started-----\n')

        since = time.time()

        train_loss, valid_loss, dice_score = 0, 0, 0

        model.train()

        bar = tq(train_loader, desc=f"Training [{e}]", postfix={"train_loss": 0.0})
        for batch, (images, masks) in enumerate(bar, 1):
            # images, masks = images.to(device), masks.to(device)

            with autocast():
                pred = model(images)
                loss = loss_function(pred, masks)

            optimizer.zero_grad()

            loss.backward()
            train_loss += loss.item() * images.size(0)

            optimizer.step()
            bar.set_postfix(ordered_dict={"train_loss": loss.item()})

        train_loss = train_loss/len(train_loader.dataset)
        train_loss_list.append(train_loss)
        mlflow.log_metric("train_loss", train_loss, step=e)

        model.eval()
        del images, masks
        with torch.no_grad():
            bar = tq(validation_loader, desc=f"Validation [{e}]", postfix={"valid_loss": 0.0, "dice_score": 0.0})
            for images, masks in bar:
                # images, masks = images.to(device), masks.to(device)

                with autocast():
                    pred = model(images)
                    loss = loss_function(pred, masks)

                if not isinstance(loss_function, SoftDiceLoss):
                    dice_coeff = dice_coefficient(pred, masks).item()
                else:
                    dice_coeff = 1-loss.item()

                valid_loss += loss.item()
                dice_score += dice_coeff
                bar.set_postfix(ordered_dict={"valid_loss":loss.item(), "dice_score":dice_coeff})
        # calculate average losses
        valid_loss = valid_loss/len(validation_loader)
        valid_loss_list.append(valid_loss)

        dice_score = dice_score/len(validation_loader)
        dice_score_list.append(dice_score)

        mlflow.log_metric("valid_loss", valid_loss, step=e)
        mlflow.log_metric("dice_score", dice_score, step=e)

        time_elapsed = time.time() - since
        scheduler.step(valid_loss)

        print(
            f"\nSummary:\n",
            f"\tEpoch: {e}/{num_epochs}\n",
            f"\tAverage train loss: {train_loss}\n",
            f"\tAverage validation loss: {valid_loss}\n",
            f"\tAverage Dice score: {dice_score}\n",
            f"\tDuration: {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s"
        )
        print(f'-----Epoch {e} finished.-----\n')



-----Epoch 1 started-----



Training [1]:   0%|          | 0/313 [00:00<?, ?it/s, train_loss=0]

Validation [1]:   0%|          | 0/16 [00:00<?, ?it/s, dice_score=0, valid_loss=0]


Summary:
 	Epoch: 1/10
 	Average train loss: 0.6809490453720093
 	Average validation loss: 0.34365207981318235
 	Average Dice score: 0.05167253687977791
 	Duration: 4m 21s
-----Epoch 1 finished.-----


-----Epoch 2 started-----



Training [2]:   0%|          | 0/313 [00:00<?, ?it/s, train_loss=0]

...


Summary:
 	Epoch: 8/10
 	Average train loss: 1.6363797616958617
 	Average validation loss: 2.184800758957863
 	Average Dice score: 0.048136952333152294
 	Duration: 4m 22s
-----Epoch 8 finished.-----


-----Epoch 9 started-----



Training [9]:   0%|          | 0/313 [00:00<?, ?it/s, train_loss=0]

Validation [9]:   0%|          | 0/16 [00:00<?, ?it/s, dice_score=0, valid_loss=0]

Epoch     9: reducing learning rate of group 0 to 1.2500e-05.

Summary:
 	Epoch: 9/10
 	Average train loss: 1.5212844702720643
 	Average validation loss: 0.5635952651500702
 	Average Dice score: 0.04749030224047601
 	Duration: 4m 25s
-----Epoch 9 finished.-----


-----Epoch 10 started-----



Training [10]:   0%|          | 0/313 [00:00<?, ?it/s, train_loss=0]

In [ ]:
images, masks = next(iter(test_loader))
images, masks = images.to(device), masks.to(device)
model.eval()
with torch.no_grad():
    out = model(images)
    print(f"Mean: {out.mean().item():.1f}, StD: {out.std().item():.1f}, Min: {out.min().item():.1f}, Max: {out.max().item():.1f}")
    # out = nn.Threshold(0.062474, 0)(out)*255
    # out = out.clamp(0., 1)

fig, axes = plt.subplots(6, 3, figsize=(15, 30))

for i in range(6):
    axes[i, 0].imshow(images[i, 0].cpu().numpy(), cmap='Greys_r')
    axes[i, 1].imshow(masks[i, 0].cpu().numpy(), cmap='Greys_r')
    axes[i, 2].imshow(out[i, 0].cpu().numpy(), cmap='Greys_r')
fig.tight_layout()

In [ ]:
model.eval()
test_loss, dice_score, f1, f1_ = 0, 0, 0, 0
with torch.no_grad():
    bar = tq(test_loader, postfix={"test_loss": 0.0, "dice_score": 0.0, "f_score": 0.0})
    for images, masks in bar:
        images, masks = images.to(device), masks.to(device)

        with autocast():
            pred = model(images)
            loss = loss_function(pred, masks)

        test_loss += loss.item()*images.size(0)
        dice_cof = dice_coefficient(pred.sigmoid(), masks).item()
        dice_score += dice_cof * pred.size(0)
        f1 = f_score(pred.sigmoid(), images).item() * images.size(0)
        f1_ += f1
        bar.set_postfix(ordered_dict={"test_loss":loss.item(), "dice_score":dice_cof, "f_score": f1})

    test_loss /= len(test_loader.dataset)
    dice_score /= len(test_loader.dataset)
    f1_ /= len(test_loader.dataset)
print(f"test_loss: {test_loss} \ndice_score: {dice_score} \nf_score: {f1}")